In [1]:
import folium
import io
from PIL import Image
import folium.features
import json
import requests
from folium.plugins import FloatImage
import base64
from PIL import Image, ImageDraw, ImageFont
from datetime import datetime

import logging
import logging.handlers



In [2]:
#Hente ned informasjon om en vegstegning

def hentDetaljerteVegmeldinger(simpleDetailsUrl):
    #Henter detaljert informasjon om vegmeldinger knyttet til hendelsen
    
    headers = {"Accept":"application/vnd.svv.v1+json;charset=utf-8","X-System-ID": "RouteMonitor - ITS seksjonen Testklient"}
    trafficMessageReq = requests.get(simpleDetailsUrl,headers=headers)
    #print(simpleDetailsUrl)
    trafficMessage = trafficMessageReq.json()
    return trafficMessage['trafficMessages']

def hentRuteOgVegmeldinger (baseurl, payload, myLogger):

    r = requests.get(baseurl, params=payload)
    #Sjekke return
    myLogger.info("Status for API request")
    myLogger.info(r.status_code)
    myLogger.info(r.reason)

    data = r.json()

    numberOfRoutesFound = len(data['routes'])
    myLogger.info(f'Number of routes found: {numberOfRoutesFound}')

    besteRuteVegstengninger = []
    besteRuteVegarbeider = []

    AktiveBesteRuteVegstengninger = []
    AktiveBesteRuteVegarbeider = []

    # Antar at første rute er beste rute før alle kadidatruter sjekkes
    # Flytter flytter viktig informasjon inn i properties for å få ekse geojson
    besteRuteIndex = 0
    besteRuteVerdi = data['routes'][0]['statistic']['totalDriveTime']

    myLogger.debug(data['routes'])

    for i in range(0,numberOfRoutesFound): #begynner på 1 fordi vi antar at første rute er beste og bare sjekker resten
        i = int(i)
        # Legger rutenavn til geojson objekt på rett måte
        antallFestures= len(data['routes'][i]['features'])
        
        for j in range(0,antallFestures):
            data['routes'][i]['features'][j]['properties']['routeName'] = data['routes'][i]['routeName']
            # Hente inn alle trafikkmeldinger
            if data['routes'][i]['features'][j]['properties']['roadFeatures']['trafficMessages']:

                for trafikkmelding in data['routes'][i]['features'][j]['properties']["roadFeatures"]["trafficMessages"]:
                    # Sjekker type trafikkmelding da ulike typer skal symboliseres ulikt
                    

                    
                    tmDetaljer = hentDetaljerteVegmeldinger(trafikkmelding['simpleDetailsUrl'])
                    for tmDetalj in tmDetaljer:
                        myLogger.debug(tmDetalj)
                        tmType = tmDetalj['type']
                        tmSituationId = tmDetalj['id']
                        tmIsActive = tmDetalj['isActiveNow']
                        tmLatitude = tmDetalj['icon']['position']['coordinates'][0]
                        tmLongitude = tmDetalj['icon']['position']['coordinates'][1]
                        tmLocationDescription = tmDetalj['descriptionOfLocation']
                        tmMessageDescription = tmDetalj['descriptionOfTrafficMessage']

                        if trafikkmelding['type'] == 'RoadClosed':
                            if tmIsActive:
                                AktiveBesteRuteVegstengninger.append(folium.Marker(
                                        icon=folium.Icon(color="red"),
                                        location=[tmLongitude,tmLatitude],
                                        popup=folium.Popup(tmType+"</br>"+"Active: "+str(tmIsActive)+"</br>"+tmLocationDescription+"</br>"+tmMessageDescription ,
                                                        max_width=150,show=showPopup) 
                                    ))
                            else:
                                besteRuteVegstengninger.append(folium.CircleMarker(
                                    radius=4, fill_color="black", fill_opacity=0.8, color="black", weight=1,
                                    location=[tmLongitude,tmLatitude],
                                    popup=folium.Popup(tmType+"</br>"+"Active: "+str(tmIsActive)+"</br>"+tmLocationDescription+"</br>"+tmMessageDescription ,
                                                        max_width=150,show=showPopup)
                                ))
                        elif trafikkmelding['type'] == 'RoadWork':
                            if tmIsActive:
                                AktiveBesteRuteVegarbeider.append(folium.CircleMarker(
                                    radius=4, fill_color="orange", fill_opacity=0.8, color="yellow", weight=1,
                                    location=[tmLongitude,tmLatitude],
                                    popup=folium.Popup(tmType+"</br>"+"Active: "+str(tmIsActive)+"</br>"+tmLocationDescription+"</br>"+tmMessageDescription ,
                                                        max_width=150,show=showPopup)
                                    ))
                            else:
                                besteRuteVegarbeider.append(folium.CircleMarker(
                                    radius=4, fill_color="gray", fill_opacity=0.8, color="black", weight=1,
                                    location=[tmLongitude,tmLatitude],
                                    popup=folium.Popup(tmType+"</br>"+"Active: "+str(tmIsActive)+"</br>"+tmLocationDescription+"</br>"+tmMessageDescription ,
                                                        max_width=150,show=showPopup)
                                    ))
                        elif trafikkmelding['type'] == 'Warning':
                            pass
                        else:
                            myLogger.warning(f"Ukjent meldingstype funnet: {tmType}")
            
        
        if data['routes'][i]['statistic']['totalDriveTime'] < besteRuteVerdi:
            besteRuteIndex = i
            besteRuteVerdi = data['routes'][i]['statistic']['totalDriveTime']
            

    myLogger.info(f"Beste rute har index: {besteRuteIndex} og heter: {data['routes'][besteRuteIndex]['routeName']} og har verdi: {besteRuteVerdi}")


    besteRuteGeoJson = {
                "type": "FeatureCollection",
                "features": data['routes'][besteRuteIndex]['features']
            }
    
    erStengt = data['routes'][besteRuteIndex]['isObstructed']
    

    return besteRuteGeoJson, AktiveBesteRuteVegstengninger, AktiveBesteRuteVegarbeider, besteRuteVegstengninger, besteRuteVegarbeider, erStengt

In [3]:
# Oppsett av logging 
LOG_FILENAME = 'Log Freight Info V2.log'

logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.DEBUG,
    datefmt='%Y-%m-%d %H:%M:%S')

formatter = logging.Formatter(fmt='%(asctime)s %(levelname)-8s %(message)s',
                                  datefmt='%Y-%m-%d %H:%M:%S')

myLogger = logging.getLogger('myLogger')
myLogger.setLevel(logging.DEBUG)
handler = logging.handlers.TimedRotatingFileHandler(LOG_FILENAME, when = 'D', interval = 1, backupCount=90)
handler.setFormatter(formatter)
myLogger.addHandler(handler)

In [4]:

# Oppsett av standard verdier for kalulasjoner
#
showPopup = True # vis popup med info om stengte veier når kartet åpnes
makeImage = True # lag bilde av kartet
makeWeb = True # Lager en hjemmeside av resultatet
height = 0 # høyde på kjøretøy, i meter
bruksklasse = "Bk_10_50" # mulige bruksklasser (sett til "" for vanlig bil): [ Bk_6_28, Bk_8_32, Bk_T8_40, Bk_T8_50, Bk_10_42, Bk_10_50, Bk_10_56, Bk_10_60 ]

baseurl = "https://www.vegvesen.no/ws/no/vegvesen/ruteplan/routingservice_v3_0/open/routingservice/api/Route/bruksklasseNormalTransport"

Trondheim = [63.42839, 10.48198]
Brekstad = [63.69380, 9.66100]
Bodo = [67.28785, 14.39415]
Oslo = [59.90813, 10.74873]

RingebuOst = [61.52614, 10.14908]
RingebuVest = [61.51426, 10.13318]

stops = f"{RingebuOst[1]},{RingebuOst[0]};{RingebuVest[1]},{RingebuVest[0]}"
myLogger.info(f'Stops on spesific route {stops}')

payload = {'stops': stops, 
           'bruksklasse': bruksklasse,
           'Height': height,
           'InputSRS': 'EPSG_4326',
           'OutputSRS': 'EPSG_4326',
           'ReturnFields':'Geometry',
            'IncludeObstructedRoutes': True}

#print(payload)

myLogger.info("################################################################")
myLogger.info("#      Starter nedlasting av strekninger                       #")
myLogger.info("################################################################")


besteRuteGeoJson, AktiveBesteRuteVegstengninger, AktiveBesteRuteVegarbeider, besteRuteVegstengninger, besteRuteVegarbeider, stengt = hentRuteOgVegmeldinger (baseurl, payload, myLogger)

print(f"Er veien stengt eller:    {stengt}")

map = folium.Map([69.1, 15.8],zoom_start=10,zoom_control=False)
g = folium.GeoJson(besteRuteGeoJson,
                style_function=lambda feature: {
                    "color": "red",
                    "weight": 2,
                    "dashArray": "5, 5",
                },
                tooltip=folium.features.GeoJsonTooltip(
            fields=['routeName'],
            aliases=['Rutenavn'])
            ).add_to(map)

folium.FitOverlays(max_zoom=12).add_to(map)

#Legger til vegstengninger og hindringer
for besteRuteVegstengning in AktiveBesteRuteVegstengninger:
    print(besteRuteVegstengning.location)
    besteRuteVegstengning.add_to(map)

for besteRuteVegstengning in besteRuteVegstengninger:
    print(besteRuteVegstengning.location)
    besteRuteVegstengning.add_to(map)

for besteRuteVegarbeid in AktiveBesteRuteVegarbeider:
    print(besteRuteVegarbeid.location)
    besteRuteVegarbeid.add_to(map)

for besteRuteVegarbeid in besteRuteVegarbeider:
    print(besteRuteVegarbeid.location)
    besteRuteVegarbeid.add_to(map)


map


2025-03-13 15:23:55 INFO     Stops on spesific route 10.14908,61.52614;10.13318,61.51426
2025-03-13 15:23:55 INFO     ################################################################
2025-03-13 15:23:55 INFO     #      Starter nedlasting av strekninger                       #
2025-03-13 15:23:55 INFO     ################################################################
2025-03-13 15:23:55 DEBUG    Starting new HTTPS connection (1): www.vegvesen.no:443
2025-03-13 15:23:56 DEBUG    https://www.vegvesen.no:443 "GET /ws/no/vegvesen/ruteplan/routingservice_v3_0/open/routingservice/api/Route/bruksklasseNormalTransport?stops=10.14908%2C61.52614%3B10.13318%2C61.51426&bruksklasse=Bk_10_50&Height=0&InputSRS=EPSG_4326&OutputSRS=EPSG_4326&ReturnFields=Geometry&IncludeObstructedRoutes=True HTTP/1.1" 200 None
2025-03-13 15:23:56 INFO     Status for API request
2025-03-13 15:23:56 INFO     200
2025-03-13 15:23:56 INFO     OK
2025-03-13 15:23:56 INFO     Number of routes found: 2
2025-03-13 15:23:56 DE

Er veien stengt eller:    True
[61.519188, 10.142356]
[61.519188, 10.142356]
[61.519188, 10.142356]
[61.519188, 10.142356]
[61.519188, 10.142356]


In [5]:
print(besteRuteVegarbeid)

print(besteRuteVegarbeid.location)

NameError: name 'besteRuteVegarbeid' is not defined